In [2]:
import colorlover as cl
from IPython.display import HTML

In [3]:
# Choose colors
my_colors = ['rgb(255,237,0)',
             'rgb(184,255,185)',
             'rgb(30,221,109)',
             'rgb(255,210,255)',
             'rgb(248,114,225)',
             'rgb(137,231,255)',
             'rgb(25, 94, 242)',
             'rgb(199, 150, 162)',
             'rgb(149, 109, 162)',
             'rgb(12, 207, 196)',
             'rgb(20, 121, 132)'
             
             ]
colors = cl.scales['11']['qual']['Paired'] + my_colors
HTML(cl.to_html( colors ))

# I. Bar plots of Sobol' indices

In [4]:
import json
import os
import brightway2 as bw
import numpy as np
import pandas as pd

# For plotting
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [7]:
path = 'generated_files//gsa_results//cge_N500/sobol_indices.json'
# path = ' '

## 1. Load data

In [8]:
with open(path) as f:
    sa_dict = json.load(f)
parameters = sa_dict['parameters']
n_parameters = len(parameters)

# Extract total index into a dictionary and dataframe
total_dict, first_dict = {}, {}
total_dict['parameters'] = parameters
first_dict['parameters'] = parameters
for k in sa_dict.keys():
    if k != 'parameters':
        total_dict[k] = sa_dict[k]['ST']
        first_dict[k] = sa_dict[k]['S1']#[abs(a) for a in all_vals_first[n_all-n_parameters:]]
total_df = pd.DataFrame(total_dict)
first_df = pd.DataFrame(first_dict)

## 2. Prepare for plotting

In [18]:
# def normalize_df(df):
#     norm_df = pd.DataFrame([],index = df.index, columns = df.columns)
#     norm_df['parameters'] = df['parameters']
#     for col in df.columns:
#         if col != 'parameters':
#             norm_df[col] = df[col] / sum(df[col])
#     return norm_df

In [19]:
def change_df_index(index):
    new_index = []
    for i in index:
        name = i.replace('_', ' ').capitalize()
        if name.lower() == 'co2 emissions':
            name = 'Direct CO2 emissions'
        elif name.lower() == 'collection pipelines':
            name = 'Collection pipelines length'
        elif 'success' in name.lower():
            name = name[:13] + 'of ' + name[13:]
        elif '0to1' in name.lower():
            name = name[:-4] + '0 ot 1'
        new_index.append(name)
    return new_index

In [20]:
def prepare_df(first_or_total):
    # choose total or first here
    if first_or_total == 'first':
        names = first_df['parameters'].tolist()
        df = first_df
    elif first_or_total == 'total':
        names = total_df['parameters'].tolist()
        df = total_df

    # Choose the order of parameters
    if 'cge' in path:
        df = df.set_index('parameters')
        new_order = ['co2_emissions',
                     'gross_power_per_well', 
                     'average_depth_of_wells',
                     'initial_harmonic_decline_rate',
                     'success_rate_primary_wells',
                     'lifetime',
                     'collection_pipelines', 
                     'installed_capacity', 
                     'capacity_factor', 
                     'auxiliary_power',
                     'specific_diesel_consumption', 
                     'specific_steel_consumption',
                     'specific_cement_consumption', 
                     'specific_drilling_mud_consumption',
                     'production_to_injection_ratio',
                     'success_rate_exploration_wells', 
                     'success_rate_makeup_wells']
    elif 'ege' in path:
        df = df.set_index('parameters')
        new_order = ['installed_capacity',
                     'average_depth_of_wells',
                     'success_rate_primary_wells',
                     'success_rate_exploration_wells',
                     'number_of_wells',
                     'lifetime',
                     'specific_diesel_consumption',
                     'specific_electricity_stimulation',
                     'water_stimulation', 
                     'auxiliary_power',
                     'specific_steel_consumption',
                     'number_of_wells_stimulated_0to1', 
                     'capacity_factor', 
                     'specific_cement_consumption',
                     'collection_pipelines',
                     'specific_drilling_mud_consumption']
        
    
    
    df = df.reindex(new_order)
    new_index = change_df_index(new_order)
    df.index = new_index
    df['index'] = np.arange(len(df))
    df = df.reset_index()
    df = df.set_index('index')
        
    return df

## 3. Bar plots

In [21]:
colors = {
    'Direct CO2 emissions': 'rgb(166,206,227)',
    'Gross power per well': 'rgb(31,120,180)', 
    'Average depth of wells': 'rgb(178,223,138)',
    'Initial harmonic decline rate': 'rgb(51,160,44)',
    'Success rate of primary wells': 'rgb(251,154,153)',
    'Lifetime': 'rgb(227,26,28)', 
    'Collection pipelines length': 'rgb(202,178,214)',
    'Installed capacity': 'rgb(106,61,154)',
    'Capacity factor': 'rgb(184,255,185)',
    'Auxiliary power': 'rgb(30,221,109)',
    'Specific steel consumption': 'rgb(248,114,225)',
    'Specific cement consumption': 'rgb(137,231,255)',
    'Specific drilling mud consumption': 'rgb(255,255,153)',
    'Production to injection ratio': 'rgb(25, 94, 242)',
    'Success rate of exploration wells': 'rgb(20, 121, 132)',
    'Success rate of makeup wells': 'rgb(199, 150, 162)',
    # missing EGE
    'Specific diesel consumption': 'rgb(255,127,0)',
    'Number of wells': 'rgb(253,191,111)',
    'Specific electricity stimulation': 'rgb(255,237,0)',
    'Water stimulation': 'rgb(255,210,255)',
    'Number of wells stimulated 0 ot 1': 'rgb(12, 207, 196)'
}

In [22]:
fig = make_subplots(rows=1, 
                    cols=2,
                    shared_yaxes=True,
                    horizontal_spacing=0.035,
                   )

flag_leg = True

for j,first_or_total in enumerate(['first', 'total']):
    
    df = prepare_df(first_or_total)
    
    ydata = df.columns[1:].tolist()
    ydata = [d.capitalize() for d in ydata]
    f = [ind for ind,yd in enumerate(ydata) if "Fossils"==yd][0]
    ydata[f] = 'Fossil resources'
    
    if 'cge' in path:
        range_ = [0,1.42]
        tickvals_ = [0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4]
        ticktext_ = [0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4]
    elif 'ege' in path:
        range_ = [0,1.03]
        tickvals_ = [0, 0.2, 0.4, 0.6, 0.8, 1]
        ticktext_ = [0, 0.2, 0.4, 0.6, 0.8, 1]

    for i in df.index:
        
        xdata = df.loc[i][1:].tolist()
        name = df.loc[i][0]
            
        fig.add_bar(name = name, 
                    x = xdata,
                    y = ydata,
                    orientation='h',
                    marker_color=colors[name],
                    row=1, 
                    col=j+1,
                    showlegend=flag_leg)
    fig.update_xaxes(range = range_,
                     tickmode = 'array', 
                     tickvals = tickvals_,
                     ticktext = ticktext_)

    fig.update_layout(barmode='stack',
                      font_size = 18,
                      width = 1600, 
                      height = 500,
                      legend_traceorder  = 'normal',
                      yaxis = dict(autorange="reversed"),
                      font_family = 'Arial',
                      margin = dict(l=0,r=0,t=0,b=0),
                      font_color = 'black',
                     )

    flag_leg = False

fig.show()

In [25]:
# Save image
path_images = "generated_plots/write_images"
if not os.path.exists(path_images):
    os.mkdir(path_images)
    
start_end = [i for i, ltr in enumerate(path) if ltr == '/']
start = start_end[-2]
end = start_end[-1]

path_fig = os.path.join(path_images, 'sobol_' + path[start+1:end] + '.png')
fig.write_image(path_fig)

# II. Plot convergence of Sobol' indices

In [ ]:
import json
import os
import brightway2 as bw
import numpy as np
import re
import pandas as pd
import itertools

from SALib.analyze import sobol

#Local files
from setup_files_gsa import *

# For plotting
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
colors = {
    'Direct CO2 emissions': 'rgb(166,206,227)',
    'Gross power per well': 'rgb(31,120,180)', 
    'Average depth of wells': 'rgb(178,223,138)',
    'Initial harmonic decline rate': 'rgb(51,160,44)',
    'Success rate of primary wells': 'rgb(251,154,153)',
    'Lifetime': 'rgb(227,26,28)', 
    'Collection pipelines length': 'rgb(202,178,214)',
    'Installed capacity': 'rgb(106,61,154)',
    'Capacity factor': 'rgb(184,255,185)',
    'Auxiliary power': 'rgb(30,221,109)',
    'Specific steel consumption': 'rgb(248,114,225)',
    'Specific cement consumption': 'rgb(137,231,255)',
    'Specific drilling mud consumption': 'rgb(255,255,153)',
    'Production to injection ratio': 'rgb(25, 94, 242)',
    'Success rate of exploration wells': 'rgb(20, 121, 132)',
    'Success rate of makeup wells': 'rgb(199, 150, 162)',
    # missing EGE
    'Specific diesel consumption': 'rgb(255,127,0)',
    'Number of wells': 'rgb(253,191,111)',
    'Specific electricity stimulation': 'rgb(255,237,0)',
    'Water stimulation': 'rgb(255,210,255)',
    'Number of wells stimulated 0 ot 1': 'rgb(12, 207, 196)'
}

## 1. Load data

In [ ]:
project = 'Geothermal'

path = 'generated_files/write_files/cge_N500'
if 'ege' in path:
    option = 'ege'
elif 'cge' in path:
    option = 'cge'

In [ ]:
scores = get_lcia_results(path)

path_sobol = os.path.join(path, 'sobol_indices.json')
with open(path_sobol) as f:
    sa_dict = json.load(f)
parameters = sa_dict['parameters']
n_parameters = len(parameters)

In [ ]:
def rename_parameters(parameters_raw):
    parameters = []
    for name in parameters_raw:
        name = name.replace('_', ' ').capitalize()
        if name.lower() == 'co2 emissions':
            name = 'Direct CO2 emissions'
        elif name.lower() == 'collection pipelines':
            name = 'Collection pipelines length'
        elif 'success' in name.lower():
            name = name[:13] + 'of ' + name[13:]
        elif '0to1' in name.lower():
            name = name[:-4] + '0 ot 1'
        parameters.append(name)
    return parameters

In [ ]:
# Choose the order of parameters
if 'cge' in path:
    parameters_ordered = [ 'co2_emissions',
                         'gross_power_per_well', 
                         'average_depth_of_wells',
                         'initial_harmonic_decline_rate',
                         'success_rate_primary_wells',
                         'lifetime',
                         'collection_pipelines', 
                         'installed_capacity', 
                         'capacity_factor', 
                         'auxiliary_power',
                         'specific_diesel_consumption', 
                         'specific_steel_consumption',
                         'specific_cement_consumption', 
                         'specific_drilling_mud_consumption',
                         'production_to_injection_ratio',
                         'success_rate_exploration_wells', 
                         'success_rate_makeup_wells' ]
elif 'ege' in path:
    parameters_ordered = ['installed_capacity',
                          'average_depth_of_wells',
                          'success_rate_primary_wells',
                          'success_rate_exploration_wells',
                          'number_of_wells',
                          'lifetime',
                          'specific_diesel_consumption',
                          'specific_electricity_stimulation',
                          'water_stimulation', 
                          'auxiliary_power',
                          'specific_steel_consumption',
                          'number_of_wells_stimulated_0to1', 
                          'capacity_factor', 
                          'specific_cement_consumption',
                          'collection_pipelines',
                          'specific_drilling_mud_consumption' ]

In [ ]:
parameters = rename_parameters(parameters)
parameters_ordered = rename_parameters(parameters_ordered)

## 2. Prepare for plotting

In [ ]:
problem, calc_second_order = setup_gsa(n_parameters)
_, methods, _, _ = setup_gt_project(project, option)

N = int(''.join(c for c in path if c.isdigit()))

In [ ]:
methods_names = [m[-1].capitalize() for m in methods]

In [ ]:
%%time
N_start = 1
N_end = N
N_step = N//100 #4032

Ns = np.arange(N_start, N_end, N_step)

first_arr = np.zeros((Ns.shape[0],len(methods),n_parameters))
total_arr = np.zeros((Ns.shape[0],len(methods),n_parameters))

for i,n in enumerate(Ns):
    
    first_methods = {}
    total_methods = {}
    num_runs = n * (n_parameters+2)
    scores_n = scores[:num_runs]
    
    for j,m in enumerate(methods_names):
        sa_dict = sobol.analyze(problem, scores_n[:,j], calc_second_order=calc_second_order)
        first = sa_dict['S1']
        total = sa_dict['ST']
        first_arr[i,j] = abs(first)
        total_arr[i,j] = total

In [ ]:
df_first = pd.DataFrame(index=methods_names, columns=parameters)
df_total = pd.DataFrame(index=methods_names, columns=parameters)

for j,m in enumerate(methods_names):
    for k,p in enumerate(parameters):
        df_first.loc[m][p] = first_arr[:,j,k]
        df_total.loc[m][p] = total_arr[:,j,k]

In [ ]:
df_first = df_first[parameters_ordered]
df_total = df_total[parameters_ordered]

## Convergence plots

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
subplots_titles = [['S1 ' + m, 'ST ' + m] for m in methods_names]


In [ ]:
# make colors transparent
opacity = 1.0
colors_opaque = {name: rgb[:3]+'a'+rgb[3:-1]+',' + str(opacity)+')' for name,rgb in colors.items()}

In [ ]:
n_rows = len(methods)
n_cols = 2
fig = make_subplots(rows=n_rows, 
                    cols=n_cols,
                    subplot_titles=flatten(subplots_titles),
                   )

In [ ]:
flag = True
for i,method in enumerate(methods_names):
    df_first_row = df_first.loc[method]
    df_total_row = df_total.loc[method]
    
    for j in range(4): 
        
        name_ = df_first_row.index[j]
        
        fig.add_trace(
            go.Scatter(x=Ns, 
                       y=df_first_row[j],
                       name=name_,
                       mode='lines+markers',
                       marker_size=8,
                       marker_symbol='star',
                       marker_opacity = opacity,
                       line=dict(color=colors_opaque[name_], width=2),
                       showlegend=False,
                      ),
            row=i+1, col=1
            )
        fig.add_trace(
            go.Scatter(x = Ns, 
                       y = df_total_row[j],
                       name = df_first_row.index[j],
                       mode='lines+markers',
                       marker_size=8,
                       marker_symbol='star',
                       marker_opacity = opacity,
                       line=dict(color=colors_opaque[name_], width=2),
                       showlegend=flag,
                      ),
            row=i+1, col=2
            )
        fig.update_layout(font_size = 18,
                          font_family = 'Arial',
                          margin = dict(l=20,r=20,t=100,b=20),
                          font_color = 'black',
                         )
    flag = False
    max_val = np.max([1, 
                      max(np.array([df_j for df_j in df_total_row]).flatten()),
                      max(np.array([df_j for df_j in df_first_row]).flatten()),
                     ])
    range_ = np.arange(0,max_val+0.5,1)
    for k in range(2):
        fig['layout']['yaxis' + str(i*2+k+1)].update(range=[-0.2, max(range_)+0.2],
                                                     tickvals=range_,
                                                     ticktext=range_)       
        
fig.update_layout(height=5000, 
                  width=1130, 
                  title_text  = "Convergence of Sobol' indices " + option.upper(),
                  title_font_color = "black",
                  title_font_size = 24,
                  title_font_family = 'Arial',
                  legend=dict(x=0.05, 
                              y=-0.01,
                              orientation="h")
                 )

for i in fig['layout']['annotations']:
    i['font']['size'] = 22
    i['font']['color'] = 'black'
    i['font']['family'] = 'Arial'
    
fig.show()

In [ ]:
# Save image
path_images = "generated_plots/write_images"
if not os.path.exists(path_images):
    os.mkdir(path_images)

path_fig = os.path.join(path_images, 'sobol_convergence_4_' + option + '.png')
fig.write_image(path_fig)